<a href="https://colab.research.google.com/github/dh610/ai-intensive2/blob/main/lab2_naive_bayes_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/ai-intensive2

!git pull

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/ai-intensive2
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 18 (delta 11), reused 0 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (18/18), 13.40 KiB | 24.00 KiB/s, done.
From https://github.com/dh610/ai-intensive2
   5f110da..76d1ed6  main       -> origin/main
Updating 5f110da..76d1ed6
Fast-forward
 lab2_LM_assignment.ipynb          |  852 ++++++++++++++++++++---------------
 lab2_naive_bayes_assignment.ipynb | 1389 +++++++++++++++++++++++++++++++++++----------------------
 2 files changed, 1339 insertions(+), 902 deletions(-)


# Lab 2 : Naive Bayes Classifier for Document Classification

@copyright:
    (c) 2023. iKnow Lab. Ajou Univ., All rights reserved.

M.S. Student: Wansik-Jo (jws5327@ajou.ac.kr)

# For assignment

- Python code의 주석 처리되어있는 부분을 구현하면 됩니다.
- MD 형식의 Cell의 [BLANK] 부분을 채우면 됩니다.
- MD 형식의 Cell의 [ANSWER] 부분 이후에 답을 작성하면 됩니다.
- 조교에게 퀴즈의 답과 함께 코드 실행 결과를 보여준 뒤, BB에 제출 후 가시면 됩니다.


## 목차
1. Document Classification task 이해하기
    - Dataset 살펴보기
    - Task 이해하기
2. Data Preprocessing
    - Dataloader 구현하기
    - Data 확인하기
    - Data preprocessing 구현하기
3. Naive Bayes Classifier 구현하기
    - 수식 이해하기
    - Vocabulary 구축하기
    - Prior 계산하기
    - Likelihood 계산하기
    - Posterior 계산하기
    - Prediction 구하기
    - Model 학습하기
    - Model 성능 평가하기
4. Naive Bayes Classifier의 여러 변형 모델 응용하기
    - 과제
---

## 1. Document Classification task 이해하기
- Dataset: NewsGroup Documents datasets
- Task: Document Classification
- Input: Document
- Output: Document Category
- Model: Naive Bayes Classifier
- Evaluation: Confusion Matrix, Accuracy

### Document Classification dataset

- URL : https://www.kaggle.com/datasets/jensenbaxter/10dataset-text-document-classification

- Kaggle 10 group dataset
- 각 Document는, 10개의 category 중 하나에 속함
- Categories : business, entertainment, food, graphics, historical, medical, politics, space, sport, technologie
- URL 또는 강의노트에서 제공되는 data를 받아서 사용

In [ ]:
#dataset load
categories = {'business', 'entertainment', 'food', 'graphics', 'historical', 'medical', 'politics', 'space', 'sport', 'technologie'}

import os

dataset = []
for category in categories:
    for filename in os.listdir('data/' + category):
        with open('data/' + category + '/' + filename, 'r') as f:
            instance = {}
            instance['text'] = f.read()
            instance['category'] = category
            dataset.append(instance)

import random
random.seed(42)
random.shuffle(dataset)
train_data = dataset[:int(len(dataset) * 0.8)]
test_data = dataset[int(len(dataset) * 0.8):]

print(len(train_data), len(test_data))

800 200


In [ ]:
#data 확인
print(train_data[0]['text'])
print(train_data[0]['category'])

#label 비율 확인
from collections import Counter
print(Counter([instance['category'] for instance in train_data]))

Pavey focuses on indoor success

Jo Pavey will miss January's View From Great Edinburgh International Cross Country to focus on preparing for the European Indoor Championships in March.

The 31-year-old was third behind Hayley Yelling and Justyna Bak in last week's European Cross Country Championships but she prefers to race on the track. "It was great winning bronze but I'm wary of injuries and must concentrate on the indoor season," she said. "Because of previous injuries I don't even run up hills in training." Pavey, who came fifth in the 5,000m at the Athens Olympics, helped the British cross country team win the team silver medal in Heringsdorf last week. She is likely to start her 3,000m season with a race in either Boston or Stuttgart at the end of January.

sport
Counter({'historical': 87, 'technologie': 83, 'food': 83, 'space': 83, 'graphics': 83, 'medical': 78, 'entertainment': 77, 'politics': 76, 'sport': 75, 'business': 75})


## 2. Data Preprocessing 구현하기

## Document raw data preprocessing

- 학습한 여러 기법을 활용하여 데이터 전처리를 진행
- Tokenizing, Stemming, Lemmatizing, Stopword 제거, Punctuation 제거, Case 변환 등 다양한 NLP pre processing 방법을 적용
- News data에 맞는 e-mail, url, phone number, number, date, time 등의 특수문자 제거 (Regex)

- !nltk library 만 사용 가능 (import nltk)
- !단 e-mail, url, phone number, number, date, time 등에 관한 처리는 regex를 사용하여 직접 구현

In [ ]:
import nltk
import re
import string

#You can use pre-defined tools in nltk, like Tokenizer, Stopword list, etc.

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stopwords = nltk.corpus.stopwords.words('english')
lemm = nltk.stem.WordNetLemmatizer()
stemmer = nltk.stem.PorterStemmer()

def clean_text(instance):
    #You should implement any pre-processing if you need.
    text = instance['text'].lower()

    #Or like here, you can use regular expression to remove any unwanted characters.
    text = re.sub(r'[^a-z\s]', '', text)

    """
    From here, you should implement any pre-process method like tokenizer, stopword list, stemming, lemmatization, etc.
    Or the regular expression above can be considered.

    text =

    """

    #print(text)

    text = nltk.word_tokenize(text)
    #print(text)
    text = [word for word in text if word not in stopwords]
    #print(text)
    text = [lemm.lemmatize(word) for word in text]
    #print(text)
    text = [stemmer.stem(word) for word in text]
    #print(text)
    text = [word for word in text if word not in string.punctuation]
    #print(text)

    instance['tokens'] = text

for instance in train_data:
    clean_text(instance)

for instance in test_data:
    clean_text(instance)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
#check
print(train_data[0]['text'])
print(train_data[0]['tokens'])

Pavey focuses on indoor success

Jo Pavey will miss January's View From Great Edinburgh International Cross Country to focus on preparing for the European Indoor Championships in March.

The 31-year-old was third behind Hayley Yelling and Justyna Bak in last week's European Cross Country Championships but she prefers to race on the track. "It was great winning bronze but I'm wary of injuries and must concentrate on the indoor season," she said. "Because of previous injuries I don't even run up hills in training." Pavey, who came fifth in the 5,000m at the Athens Olympics, helped the British cross country team win the team silver medal in Heringsdorf last week. She is likely to start her 3,000m season with a race in either Boston or Stuttgart at the end of January.

['pavey', 'focu', 'indoor', 'success', 'jo', 'pavey', 'miss', 'januari', 'view', 'great', 'edinburgh', 'intern', 'cross', 'countri', 'focu', 'prepar', 'european', 'indoor', 'championship', 'march', 'yearold', 'third', 'behin

## 3. Naive Bayes Classifier 구현하기

### Naive Bayes to Documents and Categories

document $d$ 와 category $c$ 에 대하여, posterior probability는 다음과 같이 구할 수 있다.

$$ P(c|d) = \frac{P(d|c)P(c)}{P(d)} $$

이때 $P(c)$ 를 category $c$ 의[Prior],

$P(d|c)$ 를 category $c$ 에서 document $d$ 의 [Likelihood],

$P(d)$ 를 document $d$ 의 [Prior] 라고 한다.

---

### Naive Bayes Classifier

이 때, MAP(most probable category) $c_{MAP}$ for a document $d$ 는 다음과 같이 구할 수 있다.

$$ C_{MAP} = \underset{c \in C}{\operatorname{argmax}} P(c|d) $$

그러나, $P(c|d)$ 를 직접 계산하기는 어렵다. ($P(d)$ 를 계산하기 어렵기 때문에)

따라서, 여기서 Bayes' theorem 을 사용하여 $P(c|d)$ 를 다음과 같이 바꿀 수 있다.

$$ P(c|d) = \frac{P(d|c)P(c)}{P(d)} $$

</br>

이때, $P(d)$ 는 모든 category $c$ 에 대하여 동일하므로, $P(d)$ 는 상수이다. (계산할 필요가 없다.)

따라서,

$$ C_{MAP} = \underset{c \in C}{\operatorname{argmax}} P(d|c)P(c) $$

이때, $P(d|c)$ 를 [Likelihood], $P(c)$ 를 [Prior] 라고 한다.

---

### Multinomial Naive Bayes Classifier

likelihood $P(d|c)$ 는 다음과 같이 계산할 수 있다.

$$ P(d|c) = P(w_1, w_2, \cdots, w_n|c) = \prod_{i=1}^{n} P(w_i|c) $$

이때 $w_i$ 는 $i$-th 번째 word in document $d$ 이다.

따라서, 최종 식은 다음과 같다.

$$ C_{NB} = \underset{c \in C}{\operatorname{argmax}} P(c) \prod_{i=1}^{n} P(w_i|c) $$

! 여기서 발생할 수 있는 문제점은?

[ANSWER] : 값이 너무 작아져서 컴퓨터가 값을 표현 못하는 경우가 발생(값 손실)

---

### Log Likelihood

문제는 likelihood $P(d|c)$ 가 매우 작은 값이 될 수 있다는 것이다.

따라서, likelihood 대신 log likelihood 를 사용하면 다음과 같다.

$$ C_{NB} = \underset{c \in C}{\operatorname{argmax}} P(c) \sum_{i=1}^{n} \log P(w_i|c) $$

! 여기서 발생할 수 있는 문제점은?

[ANSWER] : word가 한번도 등장하지 않을 경우 전체 값이 0이 돼버린다.

---




### Laplace smoothing

문제는 likelihood $P(d|c)$ 가 0이 될 수 있다는 것이다.

따라서 zero0-probability 를 방지하기 위하여 Laplace smoothing 을 사용한다.

$$ P(w_i|c) = \frac{count(w_i, c) + \alpha}{count(c) + \alpha \times |V|} $$

이때, $count(w_i, c)$ 는 category $c$ 에서 $w_i$ 의 개수이고

$count(c)$ 는 category $c$ 에서 word 의 총 개수,

$|V|$ 는 vocabulary 의 길이.

$\alpha$ 는 smoothing parameter 이다.

---

In [ ]:
#make vocabulary
vocabulary = set()
for instance in train_data:
    vocabulary.update(instance['tokens'])

print(categories)
print(len(vocabulary))

{'medical', 'graphics', 'politics', 'space', 'historical', 'technologie', 'entertainment', 'sport', 'food', 'business'}
152478


In [ ]:
from collections import defaultdict
import math

def train_classifier(data, categories, vocabulary):
    prior = {category: 0 for category in categories}
    likelihood = defaultdict(lambda: defaultdict(int))
    category_counts = {category: 0 for category in categories} # Initialize category_counts to track total tokens per category

    #You should implement the training process of Naive Bayes classifier.
    alpha = 1

    for instance in data:
        category = instance['category']
        for token in instance['tokens']:
            """
            Here, you should calculate the prior and likelihood.

            """
            likelihood[category][token] += 1
            category_counts[category] += 1
        prior[category] += 1

    total_instances = len(data)
    for category in categories:
        """
        Here, calculate the prior and likelihood, for each category.
        Like we have learned above, you should use Log-likelihood, and Laplace smoothing.

        prior[category] =
        for word in vocabulary:
            likelihood[category][word] =

        """
        prior[category] = math.log(prior[category]) - math.log(total_instances)
        for word in vocabulary:
            cnt_word = likelihood[category][word] + alpha
            cnt_all = category_counts[category] + alpha * len(vocabulary)
            likelihood[category][word] = math.log(cnt_word) - math.log(cnt_all)


    return prior, likelihood, category_counts

prior, likelihood, category_counts = train_classifier(train_data, categories, vocabulary)

In [ ]:
def predict(instance, prior, likelihood, categories, category_counts, vocabulary):
    score = {}

    #You should implement the prediction process of Naive Bayes classifier.

    for category in categories:
        """
        Here, you should calculate the score of each category.
        score[category] = prior[category]
        for token in instance['tokens']:
            score[catetory] +=
            You should consider the case when the token is unseen here.

        """
        score[category] = prior[category]
        for token in instance['tokens']:
            if token in vocabulary:
                score[category] += likelihood[category][token]

    return max(score, key=score.get)

print(predict(test_data[0], prior, likelihood, categories, category_counts, vocabulary))

business


In [ ]:
def evaluate(data, prior, likelihood, categories, category_counts, vocabulary):
    metrics = {
        'TP': {category: 0 for category in categories},
        'TN': {category: 0 for category in categories},
        'FP': {category: 0 for category in categories},
        'FN': {category: 0 for category in categories},
    }

    #You should implement confusion matrix here. Fill in if statement below.
    for instance in data:
        true_category = instance['category']
        predicted_category = predict(instance, prior, likelihood, categories, category_counts, vocabulary)

        for category in categories:
            if true_category == category and predicted_category == category:
                metrics['TP'][category] += 1
            if true_category != category and predicted_category != category:
                metrics['TN'][category] += 1
            if true_category != category and predicted_category == category:
                metrics['FP'][category] += 1
            if true_category == category and predicted_category != category:
                metrics['FN'][category] += 1

    precision = {category: metrics['TP'][category] / (metrics['TP'][category] + metrics['FP'][category]) for category in categories}
    recall = {category: metrics['TP'][category] / (metrics['TP'][category] + metrics['FN'][category]) for category in categories}
    accuracy = sum(metrics['TP'].values()) / len(data)

    return accuracy, precision, recall

accuracy, precision, recall = evaluate(test_data, prior, likelihood, categories, category_counts, vocabulary)
print(accuracy, precision, recall)

0.95 {'medical': 0.9545454545454546, 'graphics': 1.0, 'politics': 0.9230769230769231, 'space': 0.8888888888888888, 'historical': 1.0, 'technologie': 0.7727272727272727, 'entertainment': 1.0, 'sport': 1.0, 'food': 1.0, 'business': 1.0} {'medical': 0.9545454545454546, 'graphics': 0.9411764705882353, 'politics': 1.0, 'space': 0.9411764705882353, 'historical': 1.0, 'technologie': 1.0, 'entertainment': 0.8260869565217391, 'sport': 1.0, 'food': 0.9411764705882353, 'business': 0.92}


## 4. Naive Bayes Classifier의 여러 변형 모델 응용하기

- N-gram을 활용한 Naive Bayes Classifier (과제)

In [ ]:
import nltk
import re
import string

#You can use pre-defined tools in nltk, like Tokenizer, Stopword list, etc.

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stopwords = nltk.corpus.stopwords.words('english')
lemm = nltk.stem.WordNetLemmatizer()
stemmer = nltk.stem.PorterStemmer()

def clean_text_with_ngrams(instance, n):
    #You should implement any pre-processing if you need.
    text = instance['text'].lower()

    #Or like here, you can use regular expression to remove any unwanted characters.
    text = re.sub(r'[^a-z\s]', '', text)

    """
    From here, you should implement any pre-process method like tokenizer, stopword list, stemming, lemmatization, etc.
    Or the regular expression above can be considered.

    text =

    """

    text = nltk.word_tokenize(text)
    text = [word for word in text if word not in stopwords]
    text = [lemm.lemmatize(word) for word in text]
    text = [stemmer.stem(word) for word in text]
    text = [word for word in text if word not in string.punctuation]

    ngram_text = [tuple(text[i : i + n]) for i in range(len(text) - n)]

    instance['tokens'] = ngram_text

for instance in train_data:
    clean_text_with_ngrams(instance, 2)

for instance in test_data:
    clean_text_with_ngrams(instance, 2)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
#check
print(train_data[0]['text'])
print(train_data[0]['tokens'])

Pavey focuses on indoor success

Jo Pavey will miss January's View From Great Edinburgh International Cross Country to focus on preparing for the European Indoor Championships in March.

The 31-year-old was third behind Hayley Yelling and Justyna Bak in last week's European Cross Country Championships but she prefers to race on the track. "It was great winning bronze but I'm wary of injuries and must concentrate on the indoor season," she said. "Because of previous injuries I don't even run up hills in training." Pavey, who came fifth in the 5,000m at the Athens Olympics, helped the British cross country team win the team silver medal in Heringsdorf last week. She is likely to start her 3,000m season with a race in either Boston or Stuttgart at the end of January.

[('pavey', 'focu'), ('focu', 'indoor'), ('indoor', 'success'), ('success', 'jo'), ('jo', 'pavey'), ('pavey', 'miss'), ('miss', 'januari'), ('januari', 'view'), ('view', 'great'), ('great', 'edinburgh'), ('edinburgh', 'intern

In [ ]:
#make vocabulary
vocabulary = set()
for instance in train_data:
    vocabulary.update(instance['tokens'])

print(categories)
print(len(vocabulary))

{'medical', 'graphics', 'politics', 'space', 'historical', 'technologie', 'entertainment', 'sport', 'food', 'business'}
152478


In [ ]:


prior, likelihood, category_counts = train_classifier(train_data, categories, vocabulary)
print(predict(test_data[0], prior, likelihood, categories, category_counts, vocabulary))
accuracy, precision, recall = evaluate(test_data, prior, likelihood, categories, category_counts, vocabulary)
print(accuracy, precision, recall)

business
0.92 {'medical': 1.0, 'graphics': 1.0, 'politics': 0.8888888888888888, 'space': 0.875, 'historical': 0.8666666666666667, 'technologie': 0.7727272727272727, 'entertainment': 0.95, 'sport': 0.8928571428571429, 'food': 1.0, 'business': 1.0} {'medical': 0.8636363636363636, 'graphics': 0.7647058823529411, 'politics': 1.0, 'space': 0.8235294117647058, 'historical': 1.0, 'technologie': 1.0, 'entertainment': 0.8260869565217391, 'sport': 1.0, 'food': 1.0, 'business': 0.92}
